# Virtual ADC example
---


## Load the bitstream into the Programmable Logic

In [ ]:
# Import the X-HEEP Python class
from pynq import x_heep
# Load the X-HEEP bitstream
xh = x_heep()

In [ ]:
import sys
print(sys.version)
print(sys.path)


## Generate a random array to use as input signal 

In [ ]:
import numpy as np
# Set the seed for reproducibility (optional)
np.random.seed(46)

# Generate a random integer array
array_size = 32
rand_in = np.random.randint(0, 100, size=array_size, dtype=np.int32)

# Print the generated array
print("Generated Array:", rand_in)

# Save the array to a binary file
file_path = "/home/xilinx/x-heep-femu-sdk/sw/riscv/apps/virtual_adc_read/adc_in.bin"
data = rand_in.copy()
with open(file_path, 'wb') as file:
    data.byteswap(True).tofile(file)

## Compile the code and run the application
You can change the variable `SKIP_COMPILE` to 1 to avoid compiling each time.

In [ ]:
# This will automatically make accessible the changes made in the sdk
%reload_ext autoreload 

# Skip compiling if all you want to do is run the application
SKIP_COMPILE = 0

# Compile the application
if SKIP_COMPILE or xh.compile_app("virtual_adc_read"):
    adc = xh.init_adc_mem() # Init the ADC memory
    xh.reset_adc_mem(adc)   # Reset the ADC memory
    xh.write_adc_mem(adc)   # Write the ADC memory

    t, sf = xh.thread_start()   # Start a background thread that reads from the UART
    out, _ = xh.run_app()       # Run the application (disregarding the error output)
    xh.thread_stop(t, sf)       # Stop the background thread that reads from the UART

### Check if the CPU's output is the same as the expected result

The application is supposed to make the difference of every consecutive pair:

$y[0] = x[0]$

$y[i] = x[i] - x[i-1]$ 

$i \in (0,N]$

In [ ]:
app_out = [int(x) for x in xh.uart_data if x.strip()]

exp_out = [rand_in[i] - rand_in[i-1] for i in range(1, len(rand_in))]

if all([a == e for a,e in zip(app_out, exp_out)]): print("✅ SUCCESS")
else: print("❌ FAILED")


## Plot the input and output

In [ ]:
import matplotlib.pyplot as plt

plt.plot(rand_in)
plt.plot(app_out)

plt.legend(["Input", "Difference"])

